In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import pandas as pd
import time

In [6]:
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/122.0.0.0 Safari/537.36")
options.headless = False

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 10)

category_urls = {
    "Full Stack Programming": "https://weworkremotely.com/categories/remote-full-stack-programming-jobs",
    "Front End Programming": "https://weworkremotely.com/categories/remote-front-end-programming-jobs",
    "Back End Programming": "https://weworkremotely.com/categories/remote-back-end-programming-jobs",
    "Design": "https://weworkremotely.com/categories/remote-design-jobs",
    "DevOps and Sysadmin": "https://weworkremotely.com/categories/remote-devops-sysadmin-jobs",
    "Management and Finance":"https://weworkremotely.com/categories/remote-management-and-finance-jobs",
    "Product":"https://weworkremotely.com/categories/remote-product-jobs",
    "Customer Support":"https://weworkremotely.com/categories/remote-customer-support-jobs",
    "Sales and Marketing":"https://weworkremotely.com/categories/remote-sales-and-marketing-jobs",
    "All Other":"https://weworkremotely.com/categories/all-other-remote-jobs",
}


jobs = []

for category, url in category_urls.items():
    print(f'Scraping Category: {category}')
    driver.get(url)
    print(f"Page title: {driver.title}")
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "section.jobs li.new-listing-container")))
    except:
        print(f"No job listings loaded for {category}")
        continue
    time.sleep(2)

    try: 
        listings = driver.find_elements(By.CSS_SELECTOR, "section.jobs li.new-listing-container")
        print(f"Found {len(listings)} jobs in {category}")
        for job in listings:
            try:
                title = job.find_element(By.CLASS_NAME, "new-listing__header__title").text.strip()
                company = job.find_element(By.CLASS_NAME, "new-listing__company-name").text.strip()
                link = job.find_element(By.TAG_NAME, "a").get_attribute("href")
                postdate = job.find_element(By.CLASS_NAME, "new-listing__header__icons__date").text.strip()
                loc_elements = job.find_elements(By.CLASS_NAME, "new-listing__company-headquarters")
                loc = loc_elements[0].text.strip() if loc_elements else "N/A"
                jobs.append({
                    "title": title,
                    "company": company,
                    "category":category,
                    "date_posted":postdate,
                    "location":loc,
                    "url": link
                })
            except Exception as e:
                print(f"Error parsing job in {category}:{e}")
            
    except Exception as e:
        print(f'Error Loading {category}:{e}')

driver.quit()

wwr = pd.DataFrame(jobs)

wwr.head()

Scraping Category: Full Stack Programming
Page title: Remote Full-Stack Programming Jobs
Found 51 jobs in Full Stack Programming
Scraping Category: Front End Programming
Page title: Remote Front-End Programming Jobs
Found 10 jobs in Front End Programming
Scraping Category: Back End Programming
Page title: Remote Back-End Programming Jobs
Found 19 jobs in Back End Programming
Scraping Category: Design
Page title: Remote Design Jobs
Found 13 jobs in Design
Scraping Category: DevOps and Sysadmin
Page title: Remote DevOps and Sysadmin Jobs
Found 8 jobs in DevOps and Sysadmin
Scraping Category: Management and Finance
Page title: Remote Management and Finance Jobs
Found 20 jobs in Management and Finance
Scraping Category: Product
Page title: Remote Product Jobs
Found 9 jobs in Product
Scraping Category: Customer Support
Page title: Remote Customer Support Jobs
Found 29 jobs in Customer Support
Scraping Category: Sales and Marketing
Page title: Remote Sales and Marketing Jobs
Found 67 jobs in

,title,company,category,date_posted,location,url
0,Staff Software Engineer,BaxEnergy,Full Stack Programming,NEW,"Acireale, Italy",https://weworkremotely.com/company/baxenergy
1,Founding Engineer (Eng #4),Revenue Vessel,Full Stack Programming,NEW,Remote (United States),https://weworkremotely.com/company/revenue-vessel
2,AI Automation Expert,Contra,Full Stack Programming,3d,"San Francisco, CA",https://weworkremotely.com/company/contra
3,Remote Full-Stack TypeScript Engineer,Glama,Full Stack Programming,3d,"Miami, FL",https://weworkremotely.com/company/glama
4,Coding Tech Bootcamp - Job Guaranteed by Metana,Metana,Full Stack Programming,BOOTCAMP INFO,"No coding experience required, Land a job with...",https://metana.io/full-stack-software-engineer...


In [35]:
wwr.to_csv('jobs_raw.csv',index=False)